## Logistic Regression

In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

#### defines variables

In [2]:
W = tf.Variable(tf.zeros([5, 1]), name="weights")
b = tf.Variable(0., name="bias")

#### Reads from csv file Titanic

In [3]:
data = pd.read_csv('data/Titanic.csv')
data["Age"].fillna(0, inplace=True)
data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### gets inputs

In [4]:
def inputs():
    data.replace({"Sex": {"male": 1, "female": 0}}, inplace=True)
    # data = pd.get_dummies(data, columns=['Pclass'])
    train_data = data.sample(frac=0.8, random_state=99)
    test_data = data.loc[~data.index.isin(train_data.index), :]
    X_train = train_data.loc[:, ['Pclass', 'Sex', 'Age']]
    X_train = pd.get_dummies(X_train, columns=['Pclass'])
    X_test = test_data.loc[:, ['Pclass', 'Sex', 'Age']]
    X_test = pd.get_dummies(X_test, columns=['Pclass'])
    Y_train = train_data.loc[:, ['Survived']]
    Y_test = test_data.loc[:, ['Survived']]
    return X_train, Y_train, X_test, Y_test


X_train, Y_train, X_test, Y_test = inputs()

In [8]:
X_train.head()

,Sex,Age,Pclass_1,Pclass_2,Pclass_3
247,0,24.0,0,1,0
823,0,27.0,0,0,1
566,1,19.0,0,0,1
666,1,25.0,0,1,0
199,0,24.0,0,1,0


In [7]:
Y_train.head()

,Survived
247,1
823,1
566,0
666,0
199,0


#### Inference

In [9]:
def inference(X):
    return tf.sigmoid(tf.matmul(tf.cast(X, tf.float32), W) + b)

#### Loss function using Cross-Entropy

In [10]:
def loss(X, Y):
    Y_hat = tf.transpose(inference(X))
    return tf.reduce_sum(-1*tf.matmul(tf.cast(Y, tf.float32), tf.log(Y_hat)) + 
                         tf.matmul(tf.cast(Y - 1, tf.float32), tf.log(1-Y_hat)))

In [12]:
def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

#### Evaluating chance of survival ( greater than 0.5 will be choose as survived )

In [11]:
def evaluate(session, X, Y):
    predicted = tf.cast(inference(X) > 0.5, tf.float32)
    print(session.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32))))

In [13]:
with tf.Session() as session:

    tf.global_variables_initializer().run()

    X, Y = X_train, Y_train
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    training_steps = 1000
    for step in range(training_steps):
        session.run([train_op])
        if step % 10 == 0:
            print("loss: ", session.run([total_loss]))

    evaluate(session, X_test, Y_test)
    session.close()

loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]


loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]


loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]
loss:  [nan]


TypeError: Expected float32 passed to parameter 'y' of op 'Equal', got      Survived
0           0
11          1
13          0
20          0
29          0
34          0
35          0
38          0
39          1
41          0
45          0
47          1
52          1
55          1
59          0
60          0
62          0
67          0
68          1
73          0
77          0
83          0
88          1
94          0
95          0
104         0
108         0
114         0
122         0
123         1
..        ...
740         1
741         0
744         1
748         0
750         1
761         0
762         1
764         0
771         0
773         0
779         1
790         0
794         0
795         0
803         1
807         0
820         1
830         1
831         1
835         1
838         1
849         1
855         1
859         0
862         1
865         1
868         0
874         1
880         1
885         0

[178 rows x 1 columns] of type 'DataFrame' instead.